# Min Cut Lab
**Objectives:**
- Understand the formulation of a min cut instance, in particular
    - using infinite capacity edges
    - finding an s-t cut

## Part 1: Formulating the maximum density subgraph problem.
In the first part of this lab, we will introduce the maximum density subgraph problem.

The density of a graph is defined by the number of edges included in the graph divided by the number of nodes on the graph. Given $G = (V, E)$, $d = |E|/|V|$. In this lab, we will model the problem of finding the densest subgraph by reducing it to an instance of the Minimum Cut problem.

We will first focus on the non-optimization aspect of this problem. That is, we want to answer the question: does there exist a subset of nodes S such that the density of the subgraph it makes up is at least $\lambda$?


**Q1:** Write an equation that if satisfied, proves the existence of such a subset S. Let E(S) be the edge set induced by the nodes in S (edges going from a node in S to another node in S).

**A:** $|E(S)|/|S| \geq \lambda$

**Q2:** Rewrite the equation such that we have an equation of the form $a - b*c \geq 0$.

**A:** $|E(S)| - \lambda |S| \geq 0$

So, if we want to prove that there exists some subset of nodes S that has density at least l, we just need to find one such S that satisfies the equation above. Since the relationship is greater than or equal to, we can try to maximize the LHS such that if our maximum value is $\geq 0$ then we know we have found a set S that satisfies this property. Thus, our goal is

maximize $|E(S)| - \lambda |S|$

and see if that value is $\geq 0$.

## Part 2: Using Python

In [ ]:
from max_flow import *

# define the input graph
G = nx.DiGraph()
edgeList = [(1,2,2),(1,3,5),(2,4,1),(3,2,1),(3,5,3),(4,6,6),(5,2,1),(5,4,2),(5,6,1)]
G.add_weighted_edges_from( edgeList, 'cap' )
pos = ((0,0),(0,10),(10,20),(10,0),(30,20),(30,0),(40,10))
for i in range(1,7):
    G.nodes[i]['pos'] = pos[i]

ex = max_flow(G)

The following code will execute Ford-Fulkerson on the given graph and print out the max flow value as well as the list of nodes in the s side of the minimum cut.

In [ ]:
ex.ford_fulkerson(s=1, t=6, show=False)
print("Max flow value: " + str(ex.get_flow_value(t=6)))
ex.plot_flow()

ex.label(s=1, auto=True, show=False)
print("Minimum s-t cut: " + str(ex.get_checked_nodes()))
ex.plot_checked(residual=True)